<a href="https://colab.research.google.com/github/gabisouzaes/AnalisadorReunioesDevComGemini/blob/main/AnalisadorReunioesDevComGemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
!pip install -q google-generativeai
!pip install -q google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [125]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [126]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [127]:
# Importa as bibliotecas

# --- Importações Padrão do Python ---
import os  # Para interagir com o sistema operacional (ex: variáveis de ambiente)
import json  # Para trabalhar com dados no formato JSON

# --- Importações para Interação com APIs do Google (incluindo Google Docs) ---
# Biblioteca principal para construir serviços para APIs do Google
from googleapiclient.discovery import build

# Bibliotecas para autenticação no Google Cloud e Colab
from google.colab import auth, userdata  # 'auth' para autenticar o usuário no Colab, 'userdata' para acessar Secrets
import google.auth # Para obter as credenciais padrão após a autenticação

# --- Importações para o Google Generative AI (Gemini) ---
import google.generativeai as genai_sdk # SDK Padrão do Google Gemini, nomeado como 'genai_sdk'

# --- Importações para Exibição no Colab (Opcional, mas útil) ---
from IPython.display import Markdown, display # Para formatar e exibir saídas de forma mais rica

# --- Importação para Datas (Opcional, mas útil para registrar quando as análises são feitas) ---
from datetime import datetime

print("Todas as bibliotecas e funções necessárias foram importadas.")
print("Lembre-se de configurar sua API Key do Gemini e autenticar o usuário para o Google Docs.")

Todas as bibliotecas e funções necessárias foram importadas.
Lembre-se de configurar sua API Key do Gemini e autenticar o usuário para o Google Docs.


In [128]:
TECH_LEAD_NAME = "Gabriella"
NOME_MODELO_GEMINI_STRING = "gemini-2.0-flash"

In [129]:
# Inicializa o serviço do Google Docs
try:
    auth.authenticate_user()
    print("Usuário autenticado para serviços Google.")
    creds, project = google.auth.default()
    # Inicializar o serviço do Google Docs
    docs_service = build('docs', 'v1', credentials=creds)
    print("Serviço do Google Docs pronto para uso.")
except Exception as e:
    print(f"Erro na autenticação do Google: {e}")

Usuário autenticado para serviços Google.
Serviço do Google Docs pronto para uso.


In [131]:
PROMPT_INSIGHTS = """Você é uma assistente de IA com especialidade em identificar tendências de desenvolvimento em séries de conversas 1:1.
As reuniões analisadas são entre {nome_da_lider} (líder técnica) e um de seus liderados.
Interprete o conjunto de anotações de reuniões fornecido abaixo como um todo, refletindo a jornada de desenvolvimento deste liderado específico com {nome_da_lider}.

Anotações das Reuniões Agrupadas:
---
{textos_combinados_das_reunioes}
---

Com base em todas as anotações fornecidas, elabore um resumo com suas percepções gerais sobre os pontos de desenvolvimento do liderado. Em seu resumo, procure destacar:

* As principais áreas de foco para desenvolvimento (técnicas e comportamentais) que se mantiveram ao longo do tempo.
* Evidências claras de progresso ou, alternativamente, áreas onde o desenvolvimento pareceu estagnado ou enfrentou dificuldades.
* Desafios ou obstáculos que foram mencionados ou observados de forma recorrente.
* De 1 a 3 recomendações principais ou próximos passos para o desenvolvimento futuro do liderado.

Apresente suas percepções de forma clara, direta e concisa.
"""
def gerar_insights_de_documentos_agrupados(
    lista_ids,
    servico_docs,
    nome_lider,
    template_prompt,
    modelo_gemini_obj
):
    """Lê, combina e envia conteúdos para análise do Gemini (versão simplificada)."""
    textos_docs = []
    for doc_id in lista_ids:
        texto_lido = ler_transcricao_do_google_docs(doc_id, servico_docs)
        if texto_lido != "DOCUMENTO_VAZIO" and "ERRO" not in texto_lido:
            textos_docs.append(f"--- CONTEÚDO DOCUMENTO (ID: {doc_id}) ---\n{texto_lido}\n--- FIM DOCUMENTO (ID: {doc_id}) ---\n\n")

    textos_combinados = "".join(textos_docs)
    if not textos_combinados.strip():
        return "ERRO: Nenhum conteúdo de documento para analisar."

    prompt_final = template_prompt.format(
        nome_da_lider=nome_lider,
        textos_combinados_das_reunioes=textos_combinados
    )
    # Chamada direta à API, sem try-except para máxima simplificação aqui.
    # Erros da API irão parar o script.
    resposta = modelo_gemini_obj.generate_content(prompt_final)
    return resposta.text

# --- 6. EXECUÇÃO PRINCIPAL DA ANÁLISE ---
print("\n--- Execução Principal da Análise ---")

if docs_service and modelo_gemini_objeto:
    lista_de_document_ids = obter_lista_de_ids_google_docs()

    if lista_de_document_ids:
        print("\n   Iniciando geração de insights...")
        insights_finais = gerar_insights_de_documentos_agrupados(
            lista_de_document_ids,
            docs_service,
            TECH_LEAD_NAME,
            PROMPT_INSIGHTS,
            modelo_gemini_objeto
        )
        print("\n   --- RESULTADO DA ANÁLISE DA IA ---")
        display(Markdown(str(insights_finais)))
    else:
        print("\n   Nenhum ID de documento fornecido. Análise não realizada.")
else:
    print("\n   ERRO CRÍTICO: 'docs_service' ou 'modelo_gemini_objeto' não inicializado(s). Verifique a Seção 4.")

print("\n--- Fim do Script ---")



--- Execução Principal da Análise ---

   --- Coleta de IDs de Documentos do Google Docs ---
   Insira o ID do Documento nº 1 (ou digite 'fim' para concluir, ou pressione Enter para concluir se já adicionou algum): 1y7DvxL4VR_aQXqv8fV9b9A0nUXbhvQE4duVclH95SrU
   ID '1y7DvxL4VR_aQXqv8fV9b9A0nUXbhvQE4duVclH95SrU' adicionado à lista.
   Insira o ID do Documento nº 2 (ou digite 'fim' para concluir, ou pressione Enter para concluir se já adicionou algum): 1PUot-i8gDbY5onUDSHLPDqxNMO8Kjh_VqrNBB0-lxQw
   ID '1PUot-i8gDbY5onUDSHLPDqxNMO8Kjh_VqrNBB0-lxQw' adicionado à lista.
   Insira o ID do Documento nº 3 (ou digite 'fim' para concluir, ou pressione Enter para concluir se já adicionou algum): 19Y5asb-CN80SIjToMW-fagPQ-Gh56qRiYp5_WwJun34
   ID '19Y5asb-CN80SIjToMW-fagPQ-Gh56qRiYp5_WwJun34' adicionado à lista.
   Insira o ID do Documento nº 4 (ou digite 'fim' para concluir, ou pressione Enter para concluir se já adicionou algum): 
   Entrada vazia detectada. Coleta de IDs finalizada.

   --- C

## Percepções sobre o Desenvolvimento do Liderado

Analisando as anotações das reuniões 1:1, o liderado demonstra um desenvolvimento consistente em suas habilidades técnicas e comportamentais, com áreas de foco bem definidas e progresso mensurável ao longo do tempo.

**Áreas de Foco Consistentes:**

*   **Técnico:** Análise de dados (especialmente funil de vendas), identificação de gargalos, e otimização de processos de RevOps. Um tema recorrente é a necessidade de garantir a qualidade e consistência dos dados no CRM para análises precisas.
*   **Comportamental:** Comunicação, especificamente a capacidade de traduzir insights técnicos em informações acionáveis para diferentes stakeholders, com foco em vendas e marketing.

**Evidências de Progresso:**

*   **Comunicação:** Inicialmente, a líder apontou a necessidade de melhorar a comunicação dos resultados da análise do pipeline para a equipe de vendas. O liderado demonstrou progresso significativo ao implementar uma abordagem mais focada em insights acionáveis, resultando em melhor receptividade por parte dos stakeholders.
*   **Planejamento e Organização:** A implementação de blocos de tempo para "data hygiene" no CRM, sugerida pela líder, provou ser uma estratégia eficaz para melhorar a qualidade dos dados e otimizar o tempo de análise, mostrando a capacidade do liderado em implementar e adaptar sugestões.
*   **KPIs RevOps:** Demonstra melhora no KPI "Taxa de Adoção do Novo Processo de Qualificação de Leads (Scorecards)", indicando sucesso na condução da sessão de reforço com o time de SDRs.

**Desafios e Obstáculos:**

*   **Qualidade dos Dados:** A inconsistência e a falta de padronização dos dados no CRM (especialmente "Origem do Lead" e timestamps de mudança de estágio) surgem como um desafio recorrente, impactando a eficiência da análise e a precisão dos resultados.
*   **Atingimento de Metas:** A dificuldade em atingir a meta de "Acuracidade da Previsão de Vendas (Forecast Accuracy)" expõe uma área onde o liderado pode precisar de mais suporte para identificar e mitigar riscos em oportunidades de vendas.

**Recomendações e Próximos Passos:**

1.  **Foco na Automação da Qualidade dos Dados:** Explorar ferramentas e workflows para automatizar a validação e limpeza dos dados no CRM, reduzindo a dependência de processos manuais e garantindo a consistência das informações. Isso pode incluir a implementação de regras de validação, alertas para dados inconsistentes e integrações com outras fontes de dados.
2.  **Desenvolvimento de Habilidades de Forecasting:** Aprofundar o conhecimento sobre metodologias de previsão de vendas e análise de riscos, buscando entender os fatores que impactam a acuracidade do forecast e desenvolver estratégias para mitigar surpresas negativas.
3.  **Apresentação para Liderança:** A apresentação dos achados da análise do funil para a liderança de Vendas e Marketing é uma excelente oportunidade para o liderado demonstrar suas habilidades de comunicação e influência, consolidando seu papel como um parceiro estratégico para essas áreas. A preparação cuidadosa, com foco no impacto no revenue e no ROI das recomendações, será crucial para o sucesso dessa apresentação.



--- Fim do Script ---
